In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from alpha_vantage.fundamentaldata import FundamentalData
from alpha_vantage.timeseries import TimeSeries
import numpy as np
import pandas as pd
from pandas import DataFrame as df, Timestamp
import csv
import time
#import module for mysql 
import os
from os import path
import mysql.connector as mysql
from CalFundamental import calFund
from APICall import ApiCalling, ApiKeyDistribute
from mysqlProc import createTableSQL, insertTableSQL
from playsound import playsound


def makeFolder() :
    # define the name of the directory to be created
    path = "/Varians"
    #creating folder:
    try:
        os.makedirs(path)
    except OSError:
        print ("Directory %s already exist" % path)
    else:
        print ("Successfully created the directory %s" % path)

def stocklisting() :
    with open(r'stocklist.csv') as slist:
        stocklist = pd.read_csv(slist, header = None)    
    stocklist = stocklist.rename(columns={0 : 'company', 1 : 'market', 2 : 'ticker'})
    stocklist.ticker = stocklist.ticker.str.strip()
    return stocklist.ticker

if __name__ == "__main__":        
    #check and create folder varians as temporary storage
#     serverHost = "remotemysql.com"
#     userInput = "0NMKbNP49w"
#     mysqlPassword = "uh7eChPxHw"
#     db_name = "0NMKbNP49w"
    serverHost = "localhost"
    userInput = "root"
    mysqlPassword = "minyak23"
    db_name = "newstockmarket"
    API_KEY_TEST = ""
    makeFolder()
    stocklist = stocklisting()
    
    #creating table in mysql database, insert password accordingly
    createTableSQL(serverHost, userInput, mysqlPassword, db_name)
    
    for zticker in list (stocklist) :
        entity = str(zticker)
        print ("Stock Ticker Name : "+ entity)
    
        #reset data memory
        OV_reader = pd.DataFrame()
        ISQ_reader = pd.DataFrame()
        CFQ_reader = pd.DataFrame()
        BSQ_reader = pd.DataFrame()
        dict_table = pd.DataFrame()

        #API calling
        API_KEY_TEST = ApiKeyDistribute().randomize(API_KEY_TEST)
        e = ApiCalling(zticker, API_KEY_TEST).step_1()
        ApiCalling(zticker, API_KEY_TEST).step_2(e)
        ApiCalling(zticker, API_KEY_TEST).step_3(e)
        ApiCalling(zticker, API_KEY_TEST).step_4(e)
        ApiCalling(zticker, API_KEY_TEST).step_5(e)

        if e != 'Skip Ticker' :

            try :
                Calculation, CFQ_reader, ISQ_reader, BSQ_reader, OV_reader = calFund()
                #preparing pandas dataframe to list inputing to mysql
                insertTableSQL(serverHost, userInput, mysqlPassword, db_name, entity, Calculation, CFQ_reader, ISQ_reader, BSQ_reader, OV_reader)
                
            except TypeError as e :
                continue
            playsound('alert.wav')

Directory /Varians already exist
Stock Ticker Name : OSTK
API key is : G23MAAVKWB5TMPMV
Succsesfully Pull Data Income Statement from web
Succsesfully Pull Data Cash Flow from web
Succsesfully Pull Data Balance Sheet from web
Succsesfully Pull Data Overview from web
Succsesfully Pull Data Timeseries from web
Stock Ticker Name : CVNA
API key is : 6PE06AMZAM1MCGFX
Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.
Skip step_1
API key is : 6HZYCBP71FGOR1O5
Succsesfully Pull Data Income Statement from web
Succsesfully Pull Data Cash Flow from web
Succsesfully Pull Data Balance Sheet from web
Succsesfully Pull Data Overview from web
Succsesfully Pull Data Timeseries from web
Stock Ticker Name : CHWY
API key is : 6HZYCBP71FGOR1O5
Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 ca

TypeError: can only concatenate str (not "float") to str

In [ ]:
##Script to drop table 
import mysql.connector as mysql

def connect():
    try:
        return mysql.connect(
#             host='remotemysql.com',
#             user='0NMKbNP49w',
#             password="uh7eChPxHw",
#             database="0NMKbNP49w")
    #if mysql is local
            host='localhost',
            user='root',
            password="minyak23",
            database="newstockmarket")
    except Error as e:
        print(e)
        
#--------------------------------------------------------------------------------------------------------------------------------------------------
ftable = "stock_fundamental"
ovtable = "stock_overview"
fintable = "financial_report"
cashtable = "cashflow"
incometable = "income_statement"
balancetable = "balance_sheet"     

droptable_default = ( "DROP TABLE {table}"
                   )

db = connect()
cursor = db.cursor()

try :
    cursor.execute(droptable_default.format(table=ftable))
    cursor.execute(droptable_default.format(table=ovtable))
    cursor.execute(droptable_default.format(table=fintable))
    cursor.execute(droptable_default.format(table=cashtable))
    cursor.execute(droptable_default.format(table=incometable))
    cursor.execute(droptable_default.format(table=balancetable))    
    print ("Success Drop All Table")
except mysql.Error as err:
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)
    
db.commit()
db.close()  